# 9.2 Exercise (11-1, 11-3, 11-4)
http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT

## Exercise 11-1
Suppose one of your co-workers is expecting a baby and you are participating in an
office pool to predict the date of birth. Assuming that bets are placed during the 30th
week of pregnancy, what variables could you use to make the best prediction? You should
limit yourself to variables that are known before the birth, and likely to be available to
the people in the pool.

In [24]:
import numpy as np
import nsfg
import first
import statsmodels.formula.api as smf
import thinkstats2
import pandas as pd
# put pregnancies in data frames
live, firsts, others = first.MakeFrames()
# live births > 30 weeks
live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
# join response data set with pregnancy data set
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')
join.shape

(8884, 3331)

The next 2 chunks are code taken from the text in order to mine through variables of nsfg

In [13]:
import patsy
# altered to search for bet variables for prglngth
def GoMining(df):
    """Searches for variables that predict birth weight.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'prglngth ~ ' + name
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError, patsy.PatsyError) as e:
            continue
        
        variables.append((results.rsquared, name))

    return variables

In [19]:
import re

def ReadVariables():
    """Reads Stata dictionary files for NSFG data.

    returns: DataFrame that maps variables names to descriptions
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

def MiningReport(variables, n=30):
    """Prints variables with the highest R^2.

    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)

In [48]:
# create ordered list of variables with highest effect on preg length
variables = GoMining(join)
MiningReport(variables)

prglngth 1.0 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.8062434116139235 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
totalwgt_lb 0.12445743148120225
birthwgt_lb 0.11977307804917103 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.10372542204583302 LOW BIRTHWEIGHT - BABY 1
mosgest 0.09562431989592712 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i 0.022053775796469388 PRGLNGTH IMPUTATION FLAG
canhaver 0.00605049526819279 DF-1 PHYSICALLY DIFFICULT FOR R TO HAVE A BABY
datcon01_i 0.0058177552998763815 DATCON01 IMPUTATION FLAG
con1mar1_i 0.005546376136239206 CON1MAR1 IMPUTATION FLAG
nbrnaliv 0.004577565785532478 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
mar1con1_i 0.0031508022538583313 MAR1CON1 IMPUTATION FLAG
anynurse 0.002452024883708881 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.0023691839446681184 DURATION OF BREASTFEEDING IN WEEKS
pregend1 0.0022493894338015963 BC-1 HOW PREGNANCY ENDED - 1ST M

In [6]:
# using above, select those that have highest effects on length and for which we would reasonably have knowledge of
formula = ('prglngth ~ birthord == 1 + race==2 +  nbrnaliv>1')
results = smf.ols(formula, data = live).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     34.28
Date:                Sun, 15 May 2022   Prob (F-statistic):           5.09e-22
Time:                        11:12:06   Log-Likelihood:                -18247.
No. Observations:                8884   AIC:                         3.650e+04
Df Residuals:                    8880   BIC:                         3.653e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7617      0.039   1006.410      0.000      38.686      38.837
birthord == 1[T.True]     0.1015      0.040      2.528      0.011       0.023       0.180
race == 2[T.True]         0.1390      0.042      3.311      0.001       0.057       0.221
nbrnaliv > 1[T.True]     -1.4944      0.164     -9.086      0.000      -1.817      -1.172
==============================================================================
Omnibus:                     1587.470   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6160.751
Skew:                          -0.852   Prob(JB):                         0.00
Kurtosis:                       6.707   Cond. No.                         10.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Exercise 11-3
If the quantity you want to predict is a count, you can use Poisson regression, which is
implemented in StatsModels with a function called poisson. It works the same way as
ols and logit. As an exercise, let’s use it to predict how many children a woman has
born; in the NSFG dataset, this variable is called numbabes.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose
annual household income exceeds $75,000. How many children would you predict she
has born?

In [31]:
# create model based on the info we will be given
formula = 'numbabes ~ agepreg + totincr + C(race) + educat'
model = smf.poisson(formula, data = join).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 1.706263
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8878
Method:                           MLE   Df Model:                            5
Date:                Sun, 15 May 2022   Pseudo R-squ.:                 0.02006
Time:                        11:41:18   Log-Likelihood:                -15158.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                7.374e-132
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.5208      0.038     39.687      0.000       1.446       1.596
C(race)[T.2]    -0.1588      0.015    -10.681      0.000      -0.188      -0.130
C(race)[T.3]    -0.1245      0.025     -5.066      0.000      -0.173      -0.076
agepreg          0.0114      0.001      9.117      0.000       0.009       0.014
totincr         -0.0137      0.002     -7.241      0.000      -0.017      -0.010
educat          -0.0471      0.003    -15.549      0.000      -0.053      -0.041
================================================================================
"""

In [40]:
# create solution data frame with the info given
columns = ['agepreg', 'totincr', 'race', 'educat']
new = pd.DataFrame([[35, 14, 1, 16]], columns=columns)
# use predict() to get answer
answer = model.predict(new)
answer

0    2.648972
dtype: float64

We would predict her to have 2.65 children born, which we could round up or down since half children don't exist.

## Exercise 11-4
If the quantity you want to predict is categorical, you can use multinomial logistic regression,
which is implemented in StatsModels with a function called mnlogit. As an
exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced,
separated, or never married; in the NSFG dataset, marital status is encoded in
a variable called rmarital.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose
annual household income is about $45,000. What is the probability that she is married,
cohabitating, etc?

In [43]:
# create model on the info we will be given
formula = 'rmarital ~ agepreg + C(race) + educat + totincr'
model = smf.mnlogit(formula, data = join).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 1.110519
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8854
Method:                           MLE   Df Model:                           25
Date:                Sun, 15 May 2022   Pseudo R-squ.:                  0.1479
Time:                        11:48:48   Log-Likelihood:                -9865.8
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        3.8859      0.253     15.354      0.000       3.390       4.382
C(race)[T.2]    -0.7852      0.088     -8.906      0.000      -0.958      -0.612
C(race)[T.3]    -0.4935      0.135     -3.665      0.000      -0.757      -0.230
agepreg         -0.0712      0.008     -9.129      0.000      -0.086      -0.056
educat          -0.1725      0.019     -9.036      0.000      -0.210      -0.135
totincr         -0.1380      0.011    -12.093      0.000      -0.160      -0.116
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.2174      0.669     -0.325      0.745      -1.529       1.095
C(race)[T.2]    -0.5972      0.234     -2.549      0.011      -1.056      -0.138
C(race)[T.3]    -0.1914      0.330     -0.579      0.562      -0.839       0.456
agepreg          0.0127      0.018      0.690      0.490      -0.023       0.049
educat          -0.1006      0.049     -2.037      0.042      -0.197      -0.004
totincr         -0.3020      0.032     -9.471      0.000      -0.365      -0.240
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0477      0.238      0.201      0.841      -0.418       0.514
C(race)[T.2]    -0.3413      0.093     -3.686      0.000      -0.523      -0.160
C(race)[T.3]    -0.8581      0.169     -5.063      0.000      -1.190      -0.526
agepreg         -0.0400      0.008     -5.234      0.000      -0.055      -0.025
educat           0.1015      0.017      5.824      0.000       0.067       0.136
totincr         -0.2072      0.011    -18.073      0.000      -0.230      -0.185
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.0038      0.302      6.627      0.000       1.411       2.596
C(race)[T.2]    -1.0143      0.103     -9.802      0.000      -1.217      -0.811
C(race)[T.3]    -0.5586      0.155     -3.593      0.000      -0.863      -0.254
agepreg         -0.0598      0.009     -6.326      0.000      -0.078      -0.041
educat           0.0208      0.022      0.938      0.348      -0.023       0.064
totincr         -0.2849      0.014    -19.702      0.000      -0.313      -0.257
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        5.0581      0.246     20.533      0.000       4.575       5.541
C

In [47]:
# # create solution data frame with the info given
columns = ['agepreg', 'race', 'educat', 'totincr']
new = pd.DataFrame([[25, 2, 12,11]], columns=columns)
# use predict() to get answer
answer = model.predict(new)
answer

,0,1,2,3,4,5
0,0.780533,0.080891,0.00512,0.074122,0.026287,0.033048


Probability that woman is
- currently married: .781
- cohabiting: .081
- widowed: .005
- divorced: .074
- separated: .026
- never married: .033